# Robustness Analysis: Multiplicative vs Additive Errors

This notebook compares the effectiveness of the toggling-frame robustness objective for both multiplicative and additive errors. It's strucuted as follows:
* Imports
* Problem Setups: Here we compare the effect of the toggling frame robustness objective on additive and multiplicative errors without any variational states.
    * Base case: unitary smooth pulse problem w/o any robustness or sentivity objectives
    * Adjoint: Optimize pulses for insenstivity to a) additive error, b) multiplcative error and c) both
    * Toggle Robustness: Optimize pulses for toggle-frame robustness to a - c above
    * Adjoint * Toggle: Both for a - c
* Plot Fidelity vs. Error
* Plot Trajectories and Control pulses
* Pareto Frontiers of Fidelity vs. Robustness Penality

## Imports

In [10]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.develop(path="../../QuantumCollocation.jl")
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using Statistics
using CairoMakie
using NamedTrajectories
using TrajectoryIndexingUtils

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
   Resolving package versions...
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`


## Problem Setups
Here, we first setup the quantum systems and solve for optimal pulses. There are ten problems to solve in total (the default smooth unitary pulse case, and for adjoint only, robustness only, adjoint + robustness, there are 3 cases (additive error, mujltiplcative error, both) giving nine problems).

#### Preliminary Variables

In [11]:
# Problem parameters
T = 50
Δt = 0.5
U_goal = PAULIS.X
H_drive = PAULIS.X
Hₑ = H_drive
rob_scale = 1 / 8.0
a_bound = 0.2
dda_bound = 0.1
piccolo_opts = PiccoloOptions(verbose=false)
pretty_print(X::AbstractMatrix) = Base.show(stdout, "text/plain", X);

#### Quanutm System Setup 

In [12]:
# setup problems
H_drive_add = H_drive
∂ₑHₐ = H_drive

sys = QuantumSystem([H_drive])

varsys_add = VariationalQuantumSystem(
    [H_drive_add],
    [∂ₑHₐ] # Make robust to these adjoint states
)

VariationalQuantumSystem: levels = 2, n_drives = 1

In [13]:
H_drive_m = a -> a[1] * H_drive
H_vars_array = Function[H_drive_m]

varsys_mult = VariationalQuantumSystem(
    H_vars_array[1],
    H_vars_array,
    1
)

VariationalQuantumSystem: levels = 2, n_drives = 1

In [14]:
H_drive_b = a -> a[1] * H_drive + H_drive
# H_drift_a = a -> H_drive
H_vars_b_array = Function[H_drive_b]

varsys_both = VariationalQuantumSystem(
    H_vars_b_array[1],
    H_vars_b_array,
    1
)

VariationalQuantumSystem: levels = 2, n_drives = 1

#### Run Solvers

In [6]:
sweep_rob_loss_λ = [i for i in 0.0:5]

defaults = []
mult_adj_probs = []
add_adj_probs = []
both_adj_probs = []

for λ in sweep_rob_loss_λ

    default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; Q_t=λ)
    solve!(default, max_iter=150, print_level=5)
    push!(defaults, default)
    
    add_prob = UnitaryVariationalProblem(
            varsys_add, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            piccolo_options=piccolo_opts,
            Q_t=λ
        )
    solve!(add_prob, max_iter=100, print_level=5)
    push!(add_adj_probs, add_prob)

    mult_prob = UnitaryVariationalProblem(
            varsys_mult, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            piccolo_options=piccolo_opts,
            Q_t=λ
        )
    solve!(mult_prob, max_iter=100, print_level=5)
    push!(mult_adj_probs, mult_prob)

    both_prob = UnitaryVariationalProblem(
            varsys_both, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            piccolo_options=piccolo_opts,
            Q_t=λ
        )
    solve!(both_prob, max_iter=100, print_level=5)
    push!(both_adj_probs, both_prob)
end

    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, runnin

In [ ]:
rob_defaults = []
add_rob_probs = []
mult_rob_probs = []
both_rob_probs = []
sweep_rob_loss_λ = [i for i in 0.0:5]
for λ in sweep_rob_loss_λ

    rob_default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; activate_rob_loss=true, H_err=H_drive, Q_t=λ)
    solve!(rob_default, max_iter=150, print_level=5)
    push!(rob_defaults, rob_default)
    
    add_rob_prob = UnitaryVariationalProblem(
            varsys_add, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            activate_rob_loss=true,
            H_err=H_drive,
            Q_t=λ,
            piccolo_options=piccolo_opts
        )
    solve!(add_rob_prob, max_iter=100, print_level=5)
    push!(add_rob_probs, add_rob_prob)

    mult_rob_prob = UnitaryVariationalProblem(
            varsys_mult, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            activate_rob_loss=true,
            H_err=H_drive,
            Q_t=λ,
            piccolo_options=piccolo_opts
        )
    solve!(mult_rob_prob, max_iter=100, print_level=5)
    push!(mult_rob_probs, mult_rob_prob)

    both_rob_prob = UnitaryVariationalProblem(
            varsys_both, U_goal, T, Δt;
            variational_scales=[rob_scale],
            robust_times=[[T]],
            activate_rob_loss=true,
            H_err=H_drive,
            Q_t=λ,
            piccolo_options=piccolo_opts
        )
    solve!(both_rob_prob, max_iter=100, print_level=5)
    push!(both_rob_probs, both_rob_prob)
end

    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt
This is Ipopt version 3.14.17, running with linear solver MUMPS 5.8.0.

Number of nonzeros in equality constraint Jacobian...:     4728
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   155333

Total number of variables............................:      590
                     variables with only lower bounds:        0
                variables with 

## Plot Fidelity vs. Errors

In [ ]:
H_drive_add = H_drive

for i in length(sweep_rob_loss_λ)
    println("\nProcessing λ = $(sweep_rob_loss_λ[i])")
    f = Figure()
    ax1 = Axis(f[1, 1], title="additive, λ = $(sweep_rob_loss_λ[i])")
    ax2 = Axis(f[2, 1], title="multiplicative, λ = $(sweep_rob_loss_λ[i])")

    colors = Makie.wong_colors()

    εs = 0:0.01:.5

    # default case (smooth, non-variational, w/o toggling obj)
    default = defaults[i]
    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # adjoint-only (no toggling obj)

    # (mult)
    mult_adj_prob = mult_adj_probs[i]
    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    # (add)
    add_adj_prob = add_adj_probs[i]
    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)

    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)
    
    # (both)
    both_adj_prob = both_adj_probs[i]
    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    Legend(f[1,2], ax1, position=:lb)
    Legend(f[2,2], ax2, position=:lb)
end

In [ ]:
H_drive_add = H_drive

for i in length(sweep_rob_loss_λ)
    println("\nProcessing λ = $(sweep_rob_loss_λ[i])")
    f = Figure()
    ax1 = Axis(f[1, 1], title="additive, λ = $(sweep_rob_loss_λ[i])")
    ax2 = Axis(f[2, 1], title="multiplicative, λ = $(sweep_rob_loss_λ[i])")

    colors = Makie.wong_colors()

    εs = 0:0.01:.5

    # default case (smooth, non-variational, w/o toggling obj)
    default = defaults[i]
    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # toggling-obj-only (no variational states)

    rob_default = rob_defaults[i]
    ys = [unitary_rollout_fidelity(rob_default.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="non-variational robust", color=colors[2], linestyle=:solid)

    ys = [unitary_rollout_fidelity(rob_default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="non-variational robust", color=colors[2], linestyle=:solid)

    # Robust+adjoint (addtive)
    add_rob_prob = add_rob_probs[i]
    ys = [unitary_rollout_fidelity(add_rob_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (add)", color=colors[3], linestyle=:solid)

    ys = [unitary_rollout_fidelity(add_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (add)", color=colors[3], linestyle=:solid)

    # Robust+adjoint (mult)
    mult_rob_prob = mult_rob_probs[i]
    ys = [unitary_rollout_fidelity(mult_rob_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (mult)", color=colors[4], linestyle=:solid)

    ys = [unitary_rollout_fidelity(mult_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (mult)", color=colors[4], linestyle=:solid)

    # Robust+adjoint (both)
    both_rob_prob = both_rob_probs[i]
    ys = [unitary_rollout_fidelity(both_rob_prob.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (both)", color=colors[5], linestyle=:dash)

    ys = [unitary_rollout_fidelity(both_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (both)", color=colors[5], linestyle=:dash)


    Legend(f[1,2], ax1, position=:lb)
    Legend(f[2,2], ax2, position=:lb)
end
    f


Processing λ = 5.0


UndefVarError: UndefVarError: `defaults` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#TODO: add plots of trajectories

## Pareto Frontier Plots

### Robust (w/o adjoint)

In [ ]:
# Initialize storage for fidelities for each lambda
error_magnitudes = [x for x in range(0, stop=0.01, length=.5)]
n_lambdas = length(sweep_rob_loss_λ)
n_errors = length(error_magnitudes)
templates = [rob_defaults, add_rob_probs, mult_rob_probs, both_rob_probs]
# Store fidelities for each lambda and error combination
additive_fidelities = zeros(n_errors, n_lambdas)
multiplicative_fidelities = zeros(n_errors, n_lambdas)
obj_vals = []

# Calculate fidelities for each problem (lambda value)
for template in templates
    for (ε_idx, ε) in enumerate(error_magnitudes)
        # Calculate fidelities for each error magnitude
        
        for (λ_idx, prob) in enumerate(template)
            println("\nProcessing λ = $(sweep_rob_loss_λ[λ_idx])")
            obj_val = QuantumObjectives.FirstOrderObjective(Hₑ, prob.trajectory, [T]; Q_t=Q_t)#prob.trajectory.Ũ⃗ᵥ1[:, end]
            push!(obj_vals, obj_val)
            
            # Additive case
            add_err_sys = QuantumSystem(ε * (H_drive), H_drive_add)
            add_fidelity = unitary_rollout_fidelity(prob.trajectory, add_err_sys)
            additive_fidelities[ε_idx, λ_idx] = add_fidelity
            
            # Multiplicative case
            mult_err_sys = QuantumSystem([(1 + ε) * H_drive])
            mult_fidelity = unitary_rollout_fidelity(prob.trajectory, mult_err_sys)
            multiplicative_fidelities[ε_idx, λ_idx] = mult_fidelity
        end
    end


    # Plot 1: Additive Error - Fidelity vs Robustness for different error magnitudes
    p1 = plot(xlabel="Robustness ≡ |tr(R'R)|, where R = ∑ₜ(uₜ' Hₑ uₜ)", 
            ylabel="Fidelity",
            title="Additive Error: Fidelity vs Robustness (unitless)",
            legendfontsize=10, titlefontsize=12,
            grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3,
            legend=:topright)

    for (ε_idx, ε) in enumerate(error_magnitudes)
        plot!(p1, obj_vals, additive_fidelities[ε_idx, :],
            label="ε = $ε", 
            marker=:circle, markersize=3, linewidth=2)
    end

    # Plot 2: Multiplicative Error - Fidelity vs Robustness for different error magnitudes
    p2 = plot(xlabel="Robustness ≡ |tr(R'R)|, where R = ∑ₜ(uₜ' Hₑ uₜ)", 
            ylabel="Fidelity",
            title="Multiplicative Error: Fidelity vs Robustness (unitless)",
            legendfontsize=10, titlefontsize=12,
            grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3,
            legend=:topright)

    for (ε_idx, ε) in enumerate(error_magnitudes)
        plot!(p2, obj_vals, multiplicative_fidelities[ε_idx, :], 
            label="ε = $ε", 
            marker=:square, markersize=3, linewidth=2)
    end


    # Display all plots
    display(p1)
    display(p2)


    # Detailed results table
    println("\n=== Detailed Results Table ===")
    for (ε_idx, ε) in enumerate(error_magnitudes)
        println("\nε = $ε:")
        println("Weight λ  | Add Fidelity | Mult Fidelity")
        println("-" ^ 40)
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            println("$(lpad(round(λ, digits=4), 7)) | $(lpad(round(additive_fidelities[ε_idx, λ_idx], digits=6), 12)) | $(lpad(round(multiplicative_fidelities[ε_idx, λ_idx], digits=6), 13))")
        end
    end
end